In [110]:
import subprocess

def blame(file):
    return subprocess.run(["git", "blame", "--line-porcelain", file],
       capture_output=True,
       cwd=repo_root
    ).stdout.decode("utf8")

def git_ls_files(repo_root, dir=None):
    args = ["git", "ls-files"]
    if dir is not None:
        args.append(dir) 
        
    p = subprocess.run(args,
       capture_output=True,
       cwd=repo_root
    )
    
    if p.returncode != 0:
        return p.stderr.decode("utf8")
    return p.stdout.decode("utf8").split("\n")

def add_lines(counts, author, start, end):
    if author not in counts:
        counts[author] = 0
        
    diff = end - start
    counts[author] += diff

def count_file(f):
    print(f)
    counts = {}

    last_author_header_ln = 1
    last_header_ln = 1
    last_author_name = None

    blame_lines = blame(path.join(repo_root, f)).split("\n")

    for j, line in enumerate(blame_lines):
        if line == "":
            continue

        header_match = header_re.match(line)
        if header_match is not None:
            header_new_ln = int(header_match.group(1))
            last_header_ln = int(header_new_ln)
            continue

        author_match = author_re.match(line)
        if author_match is not None:
            author = author_match.group(1)
            if last_author_name is None:
                last_author_name = author
                continue
            

            add_lines(counts, last_author_name, last_author_header_ln, last_header_ln)
            
            last_author_name = author
            last_author_header_ln = last_header_ln
            
            
    add_lines(counts, last_author_name, last_author_header_ln, last_header_ln + 1)
    
    return counts

In [113]:
import re, pathlib, os.path as path

limit = 3
root_dir = "/home/bwhite/Projects/bryanchriswhite"
repo_names = [
    "storj",
]
author_regex = re.compile("\S+\s+\(([\w ]+)\s*?\d{4,}")

header_re = re.compile("\w{,40} \d+ (\d+)(:? (\d+))?")
author_re = re.compile("author (.+)$")

files_author_line_counts = {}


def filter_files(filename):
    ext = path.splitext(f)[1]
    return filename == "" or ext != ".go"


for name in repo_names:
    repo_root = path.join(root_dir, name)
    files = git_ls_files(repo_root=repo_root)
    i = 0
    for f in files:
        if filter_files(f):
            continue
        
        i += 1
        if i <= limit:
            author_line_counts = count_file(f)
            # print(author_line_counts)
            files_author_line_counts[f] = author_line_counts
            # print(files_author_line_counts)
                    
# files_author_line_counts

certificate/authorization/authorizations.go
certificate/authorization/authorizations_test.go
certificate/authorization/db.go


In [114]:
for f, author_line_counts in files_author_line_counts.items():
    print(f)
    for author, line_count in author_line_counts.items():
        print(str.format("    {}: {}", author, line_count))
                    

certificate/authorization/authorizations.go
    Bryan White: 174
    Jeff Wendling: 2
    Egon Elbre: 3
certificate/authorization/authorizations_test.go
    Jennifer Li Johnson: 1
    Bryan White: 168
    Egon Elbre: 56
    ethanadams: 7
    Isaac Hess: 1
    JT Olio: 2
    Stefan Benten: 1
    Jeff Wendling: 3
    Yingrong Zhao: 1
certificate/authorization/db.go
    Bryan White: 294
    Egon Elbre: 3
    Jennifer Johnson: 1


In [24]:
import pickle
# from github import Github

print(root_dir)
pickle_path = path.join(root_dir, "storj_repos.pickle")
storj_repos_file = open(pickle_path, "wb")
# apikey = open("github.apikey")
#                
# g = Github()
# org = g.get_organization("storj")
# repos = org.get_repos()
# page = repos.get_page(0)

print(pickle_path)
storj_repos_file.write(b"testing 123")
storj_repos_file.close()
# pickle.dump(page, storj_repos_file)

/home/bwhite/Projects
/home/bwhite/Projects/storj_repos.pickle


In [18]:
page = pickle.load(storj_repos_file)

[(r.name, r.created_at) for r in page]

UnsupportedOperation: read